In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime
import pandas as pd
import os
from sklearn.metrics import r2_score

def MakeDataFromText(filepath):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif line.count(":")==2:
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

In [ ]:
##########################################
############### INPUT HERE ###############

FILENAME = 'tags_91430.csv' # FILE NAME GOES HERE


#               PUT THE EPCS YOU WANT HERE AS A LIST OF STRINGS
EPC_LIST = \
[
    "460E", "B642", "8320", "9D1F", "152D", "88A5", "2B1C", "B0A5", "87B4", "0F4B",
]

##########################################
##########################################


data = MakeDataFromText(FILENAME) # FILE NAME GOES HERE

prefix = FILENAME.split('.')[0]

coeffs = [ ]

rsquared = [ ]

try:
    os.mkdir(prefix) # Make a folder that will contain images, spreadsheet, and coefficient values
except:
    pass

combined = dict()
for d in [ data, ]:
    for k,v in d.items():
        if k in combined: # if tag is already in dict
            for i in range(len(v[0])):
                if v[0][i] in combined[k]:
                    combined[k][v[0][i]].append(v[1][i])
                else:
                    combined[k][v[0][i]] = [v[1][i]]
        else:             # if tag is not in dict
            combined[k] = dict()
            for i in range(len(v[0])):
                combined[k][v[0][i]] = [v[1][i]]
for k,v in combined.items():
    for k2,v2 in v.items():
        combined[k][k2] = sum(v2)/len(v2)

In [ ]:
# MAKE PLOTS

for epc in EPC_LIST:
    for k,v in combined.items():           # For each EPC in all the readings
        if k[-4:]==epc:                    # If we have the right body part

            fig, ax = plt.subplots(figsize=(10, 6))
            
            s = sorted( v.items() )
            seconds, data, time = [ j[0].timestamp() for j in s ], [ j[1] for j in s ], [ j[0] for j in s ]

            # seconds = [ i - (seconds[0]+1) for i in seconds ] # Shift time to start at 0

            coef = np.polyfit(seconds, data, 2); 
            coeffs.append( coef )
            model = np.poly1d( coef )
            r2 = round(r2_score(data, model(seconds)), 4)
            rsquared.append(r2)

            ax.plot(seconds, data, '.', color='#3D98FF', markersize=0.75)      # PLOT SCATTER
            ax.plot(seconds, model(seconds), color='#FF0008', linewidth=1.5)  # PLOT QUADRATIC FIT

            ax.set_xticklabels( [ datetime.datetime.fromtimestamp(i).strftime('%H:%M:%S') for i in ax.get_xticks() ] )
            ax.set_xlabel("Time", fontsize=13)
            ax.set_ylabel("Temperature (C)", fontsize=13)
            ax.set_title("Tag Name: " + epc + r", $R^2$=" + str(r2), fontsize=13)

            fig.savefig(prefix + "/" + prefix + '_' + epc + '.png', dpi=300, bbox_inches='tight')

In [ ]:
# GENERATE TEXT FILE OF COEFFICIENTS

with open(prefix + "/" + prefix + '_coeffs.txt', 'w') as f:
    f.write('EPC\tAx^2\tBx\tC\tR^2\r')
    for i in range(len(coeffs)-1):
        f.write(EPC_LIST[i] + '\t' + str(coeffs[i][0]) + '\t' + str(coeffs[i][1]) + '\t' + str(coeffs[i][2]) + "\t" + str(rsquared[i]) + '\r')

In [ ]:
# GENERATE EXCEL SHEET OF DATA

list_dfs = list()
for epc in EPC_LIST:
    for k,v in combined.items():           # For each EPC in all the readings
        if k[-4:]==epc:                    # If we have the right body part
            s = sorted( v.items() )
            time, data = [ j[0] for j in s ], [ j[1] for j in s ]
            small_df = pd.DataFrame( data = {'Datetime':time, epc:data}, columns=['Datetime', epc] )
            list_dfs.append(small_df)

all_data = pd.concat([t for t in list_dfs], ignore_index=False)
all_data = all_data.set_index('Datetime')
all_data = all_data.sort_index()
all_data = all_data.groupby(level=0).sum()
all_data = all_data.groupby(all_data.index).sum()
all_data = all_data.replace(0, "-")
all_data.to_excel( prefix + "/" + prefix + "_excel" + ".xlsx")